In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [2]:
# Lectura de datos
data = pd.read_csv('iris.data',header=None)
data.columns = ['Sepal.length', 'Sepal.width', 'Petal.length', 'Petal.width', 'Species']

In [3]:
# Función de clasificación
def classificador(x, y, theta, theta_0, tipo):
    z = y*(np.dot(x, theta) + theta_0)
    if z < 1.0:
        z = -1
    else:
        z = 0
    
    if tipo == 1:
        z = z*y*x
    else:
        z = z*y
    
    return z

# Función de pérdida
def loess(X, Y, theta, theta_0, etha, lamb):
    nr, nc = X.shape
    suma_theta = 0
    suma_theta_0 = 0
    
    for i in range(nr):
        specie = Y.loc[i][0]
        # Positivo: Setosa, Negativo: el resto
        if specie == 'Iris-setosa':
            y = 1
        else:
            y = -1
        suma_theta +=  classificador(X.loc[i], y, theta, theta_0, tipo = 1)
        suma_theta_0 +=  classificador(X.loc[i], y, theta, theta_0, tipo = 2)
    
    theta = theta - etha*((1/nr)*suma_theta + lamb*theta)
    theta_0 = theta_0 - etha*(1/nr)*suma_theta_0
    
    return theta, theta_0

# Jacobiano
def jacob(X, Y, theta, theta_0, lamb):
    nr, nc = X.shape
    suma_jac = 0
    for i in range(nr):
        specie = Y.loc[i][0]
        # Positivo: Setosa, Negativo: el resto
        if specie == 'Iris-setosa':
            y = 1
        else:
            y = -1
        v = y*(np.dot(X.loc[i], theta) + theta_0)
        if v < 1.0:
            L = 1-v
        else:
            L = 0
        suma_jac += L
    
    jac = (1/nr)*suma_jac + (lamb/2)*np.dot(theta, theta)
    return jac

# Método de descenso por gradiente
def descenso_gradiente(X, Y, theta, theta_0, etha, lamb, MAX = 100, tol = 1e-4):
    t = 0
    error = 10
    while error >= tol and t < MAX:
        t += 1
        theta_act, theta_0_act = loess(X, Y, theta, theta_0, etha, lamb)
        error = abs(jacob(X, Y, theta_act, theta_0_act, lamb) - jacob(X, Y, theta, theta_0, lamb))
        print('Error: ', error)
        #print('Iteracion: ', t)
        theta = theta_act
        theta_0 = theta_0_act
    
    
    return theta_act, theta_0_act

In [ ]:
X = data[['Sepal.length', 'Sepal.width']]
Y = data[['Species']]
theta = np.array([0.0, 0.0])
theta_0 = 0.0
etha = 0.1
lamb = 0.5

In [9]:
theta_f, theta_0_f = descenso_gradiente(X, Y, theta, theta_0, etha, lamb)

Error:  0.2927315719393202
Error:  0.3977618312359833
Error:  0.3025072186141008
Error:  0.22920794439808156
Error:  0.30641394759835616
Error:  0.4006661318802681
Error:  0.2456417320719385
Error:  0.10118793185944386
Error:  0.18668542934146226
Error:  0.2951517169610838
Error:  0.3143412103044384
Error:  0.3423128248865739
Error:  0.31537388399004007
Error:  0.2974845673997071
Error:  0.30646800596605595
Error:  0.3213596620013438
Error:  0.2981806824149542
Error:  0.2811303324740674
Error:  0.2916997887845505
Error:  0.30609303137214927
Error:  0.2842406891103718
Error:  0.266734812297753
Error:  0.2784295567580175
Error:  0.2925684050446786
Error:  0.2830685618699146
Error:  0.27633847405008305
Error:  0.27730632461445515
Error:  0.2801661468653275
Error:  0.27135692485542806
Error:  0.2645501814970824
Error:  0.2661096789112679
Error:  0.2689577351121458
Error:  0.26065205076606346
Error:  0.2538237320186919
Error:  0.25581709187308044
Error:  0.2586869709368299
Error:  0.2603740

In [ ]:
jacob(X, Y, theta_f, theta_0_f, lamb)